In [78]:
import pandas as pd
import os 
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import plotly

In [47]:
urlcasos = r'https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_casos.csv'
urldead  = r'https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_fallecidos.csv'

def urlcsvread(url, target):
    dfin  = pd.read_csv(url)
    df = dfin.transpose()
    df = df.reset_index()
    
    dfout = pd.DataFrame(columns=['Comunidad', 'Fecha', target])
    for i in range(df.shape[1]-1):
        dfaux = pd.DataFrame(columns=['Comunidad', 'Fecha', target])
        dfaux['Fecha'] = pd.to_datetime(df['index'][2:], format='%d/%m/%Y')
        dfaux['Comunidad']  = df[i][1]
        dfaux[target] = np.array(df[i][2:])
        dfaux.reset_index(drop = True)
        dfout = dfout.append(dfaux)
        dfout = dfout.reset_index(drop = True)
        
    return dfin, dfout

dfincasos, dfoutcasos = urlcsvread(urlcasos, 'Casos')
dfindead, dfoutdead = urlcsvread(urldead, 'Fallecidos')

df = pd.merge(dfoutcasos, dfoutdead, on=['Comunidad', 'Fecha'])
df = df.sort_values(by=['Fallecidos'], ascending=False)

In [77]:
#fig = px.scatter(df, x="Fecha", y="Casos", color="Comunidad", size="Muertes", range_y=[0,np.max(df['Casos'])])
template = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]
fig = px.scatter(df, x="Fecha", y="Casos", color="Comunidad", size=list(df["Fallecidos"]),
                size_max=30, template=template[2], title="Casos de COVID-19 en España",
                labels={"Fecha": "Fecha", "Casos": "Casos"},log_x=False, width=1000, height=600)
for x in range(len(fig.data)):
    fig.data[x].hovertemplate = fig.data[x].hovertemplate.replace('size=', 'Fallecidos=')
    fig.data[x].hovertemplate = fig.data[x].hovertemplate.replace('Comunidad=', '<b>')
    fig.data[x].hovertemplate = fig.data[x].hovertemplate.replace('<br>Fecha', '</b><br>Fecha')
    fig.data[x].marker.sizeref = 0.5
fig.update_layout(
    xaxis_tickformat = '%d/%m/<br>%Y',
    annotations=[
        go.layout.Annotation(
            showarrow=False,
            text='Fuente de datos: datadista.com<br>Autor: pjpinzonc@gmail.com',
            xanchor='left',
            x=df['Fecha'].min(),
            yanchor='top',
            y=df["Casos"].max()
        )]
)    
fig


In [79]:
plotly.offline.plot(fig, filename='CasosCOVID19Espana_PJPinzon.html')

'CasosCOVID19Espana_PJPinzon.html'